In [2]:
import numpy as np
from DataSet import DataSet as DS

In [3]:
data = np.load("/proj/farhadda/div_E_U_2011-06-05T05-36-00_B.npz")
Bx = data["arr_0"]
By = data["arr_1"]
Bz = data["arr_2"]

In [4]:
ds1 = DS(SystemOfCoords="CAR", BaseAddress="/proj/farhadda/data/11226/Output_Sim_Omega/", Pattern="*.npz", NBlocks=20, UseBlock=0)
if(ds1.FaceX["X"].shape == Bx.shape, ds1.FaceY["Y"].shape == By.shape, ds1.FaceZ["Z"].shape == Bz.shape):
    print("Mesh size is compatibale with input data, u can proceed ")

Mesh size is compatibale with input data, u can proceed 


In [5]:
ds1.vars["Bx"] = dict()
ds1.vars["Bx"]["Location"] = "FaceX"
ds1.vars["Bx"]["val"] = Bx

ds1.vars["By"] = dict()
ds1.vars["By"]["Location"] = "FaceY"
ds1.vars["By"]["val"] = By

ds1.vars["Bz"] = dict()
ds1.vars["Bz"]["Location"] = "FaceZ"
ds1.vars["Bz"]["val"] = Bz

In [7]:
def Adummy(Coord):
    """
    Adummy returns an array with same shape as input coordinate array
    with zero values.
    """
    return np.zeros_like(Coord['X'])

ds1.Scalar("Ax", "EdgeX", Adummy)
ds1.Scalar("Ay", "EdgeY", Adummy)
ds1.Scalar("Az", "EdgeZ", Adummy)

First Step:
-----------
\begin{eqnarray}
A_x(x, y_1, z_2) &=& 0 \\
A_y(x_1, y, z_2) &=& 0
\end{eqnarray}

Second Step:
------------
\begin{eqnarray}
A_x(x_i, y_{j+1/2}, z_2) =   A_x(x_i, y_{j-1/2}, z_2) - \frac{\Delta y}{2} B_z(x_i, y_j, z_2) \\
A_y(x_{i+1/2}, y_j, z_2) =   A_y(x_{i-1/2}, y_j, z_2) + \frac{\Delta x}{2} B_z(x_i, y_j, z_2)
\end{eqnarray}

Final Step:
------------
\begin{eqnarray}
A_x(x, y, z_{k-1}) = A_x(x, y, z_k) - \int_{-\Delta z}^0 dz^* B_y(x, y, z^*+z_k) \\
\end{eqnarray}
and same goes for $A_{y}$-comp

In [8]:
## First Step, these values are zero, just to emphasis
ds1.vars["Ax"]["val"][:, 0, -1] = 0.0
ds1.vars["Ay"]["val"][0, :, -1] = 0.0

## Second Step, Assuming dy and dx are uniform @ upper boundery
dx = ds1.geo["x"][1:] - ds1.geo["x"][:-1]
dy = ds1.geo["y"][1:] - ds1.geo["y"][:-1]
dz = ds1.geo["z"][1:] - ds1.geo["z"][:-1]
for i in range(dx.size):
    for j in range(dy.size):
        ds1.vars["Ax"]["val"][i, j+1, -1] = ds1.vars["Ax"]["val"][i, j, -1] - 0.5 * dy[j] * ds1.vars["Bz"]["val"][i, j, -1]
    
for j in range(dy.size):
    for i in range(dx.size):
        ds1.vars["Ay"]["val"][i+1, j, -1] = ds1.vars["Ay"]["val"][i, j, -1] - 0.5 * dx[i] * ds1.vars["Bz"]["val"][i, j, -1]

## Final Step,
for k in range(dz.size - 2, -1, -1):
    for i in range(dx.size):
        for j in range(dy.size + 1):
            ds1.vars["Ax"]["val"][i, j, k] = ds1.vars["Ax"]["val"][i, j, k+1] - dz[k] * ds1.vars["By"]["val"][i, j, k+1]

    for j in range(dy.size):
        for i in range(dx.size + 1):
            #TODO :: is this equation ---<(signs)>--- correct?
            ds1.vars["Ay"]["val"][i, j, k] = ds1.vars["Ay"]["val"][i, j, k+1] + dz[k] * ds1.vars["Bx"]["val"][i, j, k+1]
            
ds1.Write2HDF5("/proj/farhadda/FirstTry.h5", databasename="Timestep_0")

In [ ]:
ds2 = DS(SystemOfCoords="CAR", BaseAddress="/proj/farhadda/data/11226/Output_Sim_Omega/", Pattern="*.npz", NBlocks=20, UseBlock=0)
ds1.ToNewMesh(ds2, "Ax", "EdgeX")
ds1.ToNewMesh(ds2, "Ay", "EdgeY")
ds2.Scalar("Az", "EdgeZ", Adummy)

ds2.CurlEdgeToFace("Ax", "Ay", "Az", "BXf", "BYf", "BZf")
ds2.DivFace("BXf", "BYf", "BZf", "DivBf")  # using A on Edges and B on Cells
ds2.CurlFaceToEdge("BXf", "BYf", "BZf", "JXe", "JYe", "JZe")
ds2.AvgFaceToCell("BXf", "BYf", "BZf", "BXc", "BYc", "BZc")
ds2.AvgEdgeToCell("JXe", "JYe", "JZe", "JXc", "JYc", "JZc")
ds2.Cross("JXc", "JYc", "JZc", "BXc", "BYc", "BZc", "LFXc", "LFYc", "LFZc")
ds2.Write2HDF5("SecondDataSet.h5")